# 基于Transformers的NER实战

## step1 导入包

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

import evaluate # evaluate=0.4.3

## step2 加载数据集（加载数据集，查看数据特征）

In [ ]:
ner_datasets = load_dataset('peoples_daily_ner', cache_dir='./data')
ner_datasets

C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:1461: FutureWarning: The repository for peoples_daily_ner contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/peoples_daily_ner
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [3]:
ner_datasets.keys()
print(ner_datasets['train'][0]) # 查看数据
ner_datasets['train'].features # 查看数据的特征

label_list = ner_datasets['train'].features["ner_tags"].feature.names
label_list

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

## step3 数据集预处理

In [4]:
## 构建tokenizer,并进行datasets的构建，进行dataloader构建
tokenizer = AutoTokenizer.from_pretrained('D:/AI/pretrain_model/models--hfl--chinese-macbert-base')

In [5]:
tokenizer(ner_datasets['train'][0]['tokens'], is_split_into_words=True)

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
res = tokenizer("海豹比赛地点在厦门与金门之间的海域")
res

{'input_ids': [101, 3862, 6501, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
res.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, None]

In [8]:
res.word_ids(0)

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, None]

In [9]:
print(ner_datasets['train'][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}


In [10]:
## 借助word_ids构建NER的标签
def process_function(examples):
    # truncation 允许截断，这里的截断是按照一定的规则进行截断，而不是直接截断
    tokenized_examples = tokenizer(examples['tokens'], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_examples.word_ids(i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                # word_ids是词的下标index，label是ner_tags中的标签，就是取对应token的标签
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels
    return tokenized_examples

In [11]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/20865 [00:00<?, ? examples/s]

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

Map:   0%|          | 0/4637 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [12]:
print(tokenized_datasets['train'][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


## step4 创建模型

In [15]:
model = AutoModelForTokenClassification.from_pretrained('D:/AI/pretrain_model/models--hfl--chinese-macbert-base', num_labels=len(label_list)) # 模型默认是二分类，需要用num_labels指定分类的个数

Some weights of BertForTokenClassification were not initialized from the model checkpoint at D:/AI/pretrain_model/models--hfl--chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model.config.num_labels

7

## step5 创建评估函数

In [17]:
seqeval = evaluate.load("seqeval_metric.py")
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [25]:
import numpy as np

def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels) 
    ]

    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")

    return {
        "f1": result["overall_f1"]
    }
    

## step6 配置训练参数

In [28]:
args = TrainingArguments(
    output_dir='model_for_ner',
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=1
)

## step7 创建训练器

In [29]:
trainer = Trainer(
    model = model,
    args = args,
    tokenizer = tokenizer,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

## step8 模型训练

In [30]:
trainer.train()

  0%|          | 0/5217 [00:00<?, ?it/s]

{'loss': 0.0201, 'grad_norm': 0.04536016285419464, 'learning_rate': 4.952079739313782e-05, 'epoch': 0.01}
{'loss': 0.0224, 'grad_norm': 0.08559268712997437, 'learning_rate': 4.904159478627564e-05, 'epoch': 0.02}
{'loss': 0.0295, 'grad_norm': 0.010819493792951107, 'learning_rate': 4.8562392179413456e-05, 'epoch': 0.03}
{'loss': 0.0197, 'grad_norm': 4.265042781829834, 'learning_rate': 4.808318957255128e-05, 'epoch': 0.04}
{'loss': 0.0156, 'grad_norm': 0.798666775226593, 'learning_rate': 4.76039869656891e-05, 'epoch': 0.05}
{'loss': 0.0125, 'grad_norm': 0.00754921417683363, 'learning_rate': 4.712478435882691e-05, 'epoch': 0.06}
{'loss': 0.0143, 'grad_norm': 2.220876693725586, 'learning_rate': 4.6645581751964735e-05, 'epoch': 0.07}
{'loss': 0.0123, 'grad_norm': 1.3952579498291016, 'learning_rate': 4.616637914510255e-05, 'epoch': 0.08}
{'loss': 0.0162, 'grad_norm': 3.518247127532959, 'learning_rate': 4.568717653824037e-05, 'epoch': 0.09}
{'loss': 0.0216, 'grad_norm': 1.6854336261749268, 'le

  0%|          | 0/19 [00:00<?, ?it/s]

{'eval_loss': 0.02287866175174713, 'eval_f1': 0.951490999038065, 'eval_runtime': 10.9232, 'eval_samples_per_second': 212.301, 'eval_steps_per_second': 1.739, 'epoch': 1.0}
{'train_runtime': 438.6879, 'train_samples_per_second': 47.562, 'train_steps_per_second': 11.892, 'train_loss': 0.01623696371276008, 'epoch': 1.0}


TrainOutput(global_step=5217, training_loss=0.01623696371276008, metrics={'train_runtime': 438.6879, 'train_samples_per_second': 47.562, 'train_steps_per_second': 11.892, 'train_loss': 0.01623696371276008, 'epoch': 1.0})

## step9 模型推理

In [34]:
from transformers import pipeline

In [35]:
model.config

BertConfig {
  "_name_or_path": "D:/AI/pretrain_model/models--hfl--chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 1

In [36]:
model.config.id2label = {idx:label  for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "D:/AI/pretrain_model/models--hfl--chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type

In [38]:
ner_pipeline = pipeline(task='token-classification', model=model, tokenizer=tokenizer, device=0)

In [41]:
res = ner_pipeline("我在深圳福田上班，我老婆在车公庙上班")
print(res)

[{'entity': 'B-LOC', 'score': 0.9991673, 'index': 3, 'word': '深', 'start': 2, 'end': 3}, {'entity': 'I-LOC', 'score': 0.9994618, 'index': 4, 'word': '圳', 'start': 3, 'end': 4}, {'entity': 'B-LOC', 'score': 0.9979067, 'index': 5, 'word': '福', 'start': 4, 'end': 5}, {'entity': 'I-LOC', 'score': 0.9992625, 'index': 6, 'word': '田', 'start': 5, 'end': 6}, {'entity': 'B-LOC', 'score': 0.99845827, 'index': 14, 'word': '车', 'start': 13, 'end': 14}, {'entity': 'I-LOC', 'score': 0.9983924, 'index': 15, 'word': '公', 'start': 14, 'end': 15}, {'entity': 'I-LOC', 'score': 0.9985978, 'index': 16, 'word': '庙', 'start': 15, 'end': 16}]


In [44]:
ner_pipeline2 = pipeline(task='token-classification', model=model, tokenizer=tokenizer, device=0, aggregation_strategy='simple')
res2 = ner_pipeline2("小明在北京上班")
res2

[{'entity_group': 'PER',
  'score': 0.9992747,
  'word': '小 明',
  'start': 0,
  'end': 2},
 {'entity_group': 'LOC',
  'score': 0.99945104,
  'word': '北 京',
  'start': 3,
  'end': 5}]

In [48]:
ner_res = {}
intput_data = "小明在北京上班"
for r in res2:
    if r['entity_group'] not in ner_res:
        ner_res[r['entity_group']] = []
    ner_res[r['entity_group']].append(intput_data[r["start"]:r["end"]])

print(ner_res)

{'PER': ['小明'], 'LOC': ['北京']}
